# 향후 판매량 예측 경진대회 베이스라인 모델
- [향후 판매량 예측 경진대회 링크](https://www.kaggle.com/c/competitive-data-science-predict-future-sales)
- [베인스라인 코드 참고 링크](https://www.kaggle.com/dkomyagin/predict-future-sales-lightgbm-framework)

In [1]:
# ch6/predict_future_sales_baseline.ipynb

import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings(action='ignore') # 경고 문구 생략

# 데이터 경로
data_path = '/kaggle/input/competitive-data-science-predict-future-sales/'

sales_train = pd.read_csv(data_path + 'sales_train.csv')
shops = pd.read_csv(data_path + 'shops.csv')
items = pd.read_csv(data_path + 'items.csv')
item_categories = pd.read_csv(data_path + 'item_categories.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

## 피처명 한글로 변경

In [2]:
sales_train = sales_train.rename(columns={'date': '날짜', 
                                          'date_block_num': '날짜ID',
                                          'shop_id': '상점ID',
                                          'item_id': '상품ID',
                                          'item_price': '판매가',
                                          'item_cnt_day': '판매량'})

sales_train.head()

,날짜,날짜ID,상점ID,상품ID,판매가,판매량
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [3]:
shops = shops.rename(columns={'shop_name': '상점명',
                              'shop_id': '상점ID'})

shops.head()

,상점명,상점ID
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [4]:
items = items.rename(columns={'item_name': '상품명',
                              'item_id': '상품ID',
                              'item_category_id': '상품범주ID'})

items.head()

,상품명,상품ID,상품범주ID
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [5]:
item_categories = item_categories.rename(columns=
                                         {'item_category_name': '상품범주명',
                                          'item_category_id': '상품범주ID'})

item_categories.head()

,상품범주명,상품범주ID
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [6]:
test = test.rename(columns={'shop_id': '상점ID',
                            'item_id': '상품ID'})

test.head()

,ID,상점ID,상품ID
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


## 데이터 형변환

In [7]:
def downcast(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('{:.1f}% 압축됨'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [8]:
all_df = [sales_train, shops, items, item_categories, test]
for df in all_df:
    df = downcast(df)

62.5% 압축됨
38.6% 압축됨
54.2% 압축됨
39.9% 압축됨
70.8% 압축됨


## 데이터 준비

### 데이터 조합 생성

In [9]:
from itertools import product

train = []
# 날짜ID, 상점ID, 상품ID 조합 생성 ---①
for i in sales_train['날짜ID'].unique():
    all_shop = sales_train.loc[sales_train['날짜ID']==i, '상점ID'].unique()
    all_item = sales_train.loc[sales_train['날짜ID']==i, '상품ID'].unique()
    train.append(np.array(list(product([i], all_shop, all_item))))

idx_features = ['날짜ID', '상점ID', '상품ID']
# 리스트 타입인 train을 DataFrame 타입으로 변환 ---②
train = pd.DataFrame(np.vstack(train), columns=idx_features)

train

,날짜ID,상점ID,상품ID
0,0,59,22154
1,0,59,2552
2,0,59,2554
3,0,59,2555
4,0,59,2564
...,...,...,...
10913845,33,21,7635
10913846,33,21,7638
10913847,33,21,7640
10913848,33,21,7632


### 타깃 값을 월간 총 판매량으로 변경

In [10]:
# idx_features를 기준으로 그룹화해 판매량 합 구하기 ---①
group = sales_train.groupby(idx_features).agg({'판매량': 'sum'})
# 인덱스 재설정
group = group.reset_index()
# 피처명을 '판매량'에서 '총 판매량'으로 변경
group = group.rename(columns={'판매량': '총 판매량'})

group

,날짜ID,상점ID,상품ID,총 판매량
0,0,0,32,6
1,0,0,33,3
2,0,0,35,1
3,0,0,43,1
4,0,0,51,2
...,...,...,...,...
1609119,33,59,22087,6
1609120,33,59,22088,2
1609121,33,59,22091,1
1609122,33,59,22100,1


In [11]:
# train과 group 병합하기
train = train.merge(group, on=idx_features, how='left')

train

,날짜ID,상점ID,상품ID,총 판매량
0,0,59,22154,1.0
1,0,59,2552,NaN
2,0,59,2554,NaN
3,0,59,2555,NaN
4,0,59,2564,NaN
...,...,...,...,...
10913845,33,21,7635,NaN
10913846,33,21,7638,NaN
10913847,33,21,7640,NaN
10913848,33,21,7632,NaN


In [12]:
import gc

# group 가비지 컬렉션
del group
gc.collect();

### 테스트 데이터 이어붙이기

In [13]:
test['날짜ID'] = 34

In [14]:
train

,날짜ID,상점ID,상품ID,총 판매량
0,0,59,22154,1.0
1,0,59,2552,NaN
2,0,59,2554,NaN
3,0,59,2555,NaN
4,0,59,2564,NaN
...,...,...,...,...
10913845,33,21,7635,NaN
10913846,33,21,7638,NaN
10913847,33,21,7640,NaN
10913848,33,21,7632,NaN


In [15]:
test

,ID,상점ID,상품ID,날짜ID
0,0,5,5037,34
1,1,5,5320,34
2,2,5,5233,34
3,3,5,5232,34
4,4,5,5268,34
...,...,...,...,...
214195,214195,45,18454,34
214196,214196,45,16188,34
214197,214197,45,15757,34
214198,214198,45,19648,34


In [16]:
# train과 test 이어붙이기
all_data = pd.concat([train, test.drop('ID', axis=1)],
                     ignore_index=True,
                     keys=idx_features)
# 결측값을 0으로 대체
all_data = all_data.fillna(0)

all_data

,날짜ID,상점ID,상품ID,총 판매량
0,0,59,22154,1.0
1,0,59,2552,0.0
2,0,59,2554,0.0
3,0,59,2555,0.0
4,0,59,2564,0.0
...,...,...,...,...
11128045,34,45,18454,0.0
11128046,34,45,16188,0.0
11128047,34,45,15757,0.0
11128048,34,45,19648,0.0


### 나머지 데이터 병합

In [17]:
# 나머지 데이터 병합
all_data = all_data.merge(shops, on='상점ID', how='left')
all_data = all_data.merge(items, on='상품ID', how='left')
all_data = all_data.merge(item_categories, on='상품범주ID', how='left')

# 데이터 형변환
all_data = downcast(all_data)

# 가비지 컬렉션
del shops, items, item_categories
gc.collect();

26.4% 압축됨


In [18]:
all_data.head()

,날짜ID,상점ID,상품ID,총 판매량,상점명,상품명,상품범주ID,상품범주명
0,0,59,22154,1,"Ярославль ТЦ ""Альтаир""",ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray
1,0,59,2552,0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил
2,0,59,2554,0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил
3,0,59,2555,0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства
4,0,59,2564,0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE Perihelion: Live In Concert DVD (К...,59,Музыка - Музыкальное видео


In [19]:
all_data = all_data.drop(['상점명', '상품명', '상품범주명'], axis=1)

In [20]:
# 훈련 데이터 (피처)
X_train = all_data[all_data['날짜ID'] < 33]
X_train = X_train.drop(['총 판매량'], axis=1)
# 검증 데이터 (피처)
X_valid = all_data[all_data['날짜ID'] == 33]
X_valid = X_valid.drop(['총 판매량'], axis=1)
# 테스트 데이터 (피처)
X_test = all_data[all_data['날짜ID'] == 34]
X_test = X_test.drop(['총 판매량'], axis=1)

# 훈련 데이터 (타깃 값)
y_train = all_data[all_data['날짜ID'] < 33]['총 판매량']
y_train = y_train.clip(0, 20) # 타깃 값을 0~20로 제한
# 검증 데이터 (타깃 값)
y_valid = all_data[all_data['날짜ID'] == 33]['총 판매량']
y_valid = y_valid.clip(0, 20)

In [21]:
del all_data
gc.collect();

## 베이스라인 모델 훈련 및 제출

In [22]:
import lightgbm as lgb

# lgb 하이퍼 파라미터
params = {'metric': 'rmse',
          'num_leaves': 255,
          'learning_rate': 0.01,
          'force_col_wise': True,
          'random_state': 50}

# 범주형 피처 설정
cat_features = ['상점ID', '상품범주ID']

# lgb 훈련 및 검증 데이터세트
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)

# LightGBM 모델 훈련
lgb_model = lgb.train(params=params,
                      train_set=dtrain,
                      num_boost_round=500,
                      valid_sets=dvalid,
                      categorical_feature=cat_features,
                      verbose_eval=50)  

[LightGBM] [Info] Total Bins 427
[LightGBM] [Info] Number of data points in the train set: 10675678, number of used features: 4
[LightGBM] [Info] Start training from score 0.299125
[50]	valid_0's rmse: 1.06765
[100]	valid_0's rmse: 1.03754
[150]	valid_0's rmse: 1.0249
[200]	valid_0's rmse: 1.01814
[250]	valid_0's rmse: 1.01409
[300]	valid_0's rmse: 1.01057
[350]	valid_0's rmse: 1.00801
[400]	valid_0's rmse: 1.00604
[450]	valid_0's rmse: 1.00506
[500]	valid_0's rmse: 1.00408


In [23]:
preds = lgb_model.predict(X_test).clip(0,20)

submission['item_cnt_month'] = preds
submission.to_csv('submission.csv', index=False)

In [24]:
del X_train, y_train, X_valid, y_valid, X_test, lgb_model, dtrain, dvalid
gc.collect();